In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.types import TimestampNTZType

import logging
import shutil

KAFKA_BROKER = 'localhost:9092'
KAFKA_TOPIC_NAME = 'stock'

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# create spark session
def create_spark_connection():
    spark = None

    try:
        spark = SparkSession \
                .builder \
                .appName("StockPriceAverage") \
                .master("local[*]") \
                .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.0,"+
                        "org.apache.kafka:kafka-clients:3.5.1,"+
                        "org.apache.spark:spark-streaming-kafka-0-10_2.13:3.5.1,"+
                        "org.apache.commons:commons-pool2:2.8.0,"+
                        "org.apache.spark:spark-token-provider-kafka-0-10_2.13:3.5.1") \
                .config("spark.executor.memory", "512M") \
                .config("spark.executor.cores", "1") \
                .config("spark.driver.cores", "1") \
                .config("spark.driver.memory", "512M") \
                .getOrCreate()

        spark.sparkContext.setLogLevel("ERROR")
        logging.info("Spark connection created successfully!")
    except Exception as e:
        logging.error(f"Couldn't create the spark session due to exception {e}")

    return spark

# spark connects to kafka
def connect_to_kafka(spark_conn):
    spark_df = None
    try:
        spark_df = spark_conn.readStream \
                    .format("kafka") \
                    .option("kafka.bootstrap.servers", KAFKA_BROKER) \
                    .option("subscribe", KAFKA_TOPIC_NAME) \
                    .load()
        logging.info("kafka dataframe created successfully")
    except Exception as e:
        logging.warning(f"kafka dataframe could not be created because: {e}")

    return spark_df

# data processing
def process(value_df):
    df = value_df.select("time", "open", "close")
    df = df.withColumn("open_close_price", round((col("close") - col("open")), 2))
    result_df = df.select("time", "open_close_price")
    result_df.printSchema()

    return result_df

# Save processed data to data folder
# def save(df_batch, batch_id):
#     df_batch.write.parquet("data", mode="append")
#     df_batch.show()
import os
def save(df_batch, batch_id):
    output_dir = "data"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    df_batch.write.parquet(output_dir, mode="append")
    df_batch.show()

# Remove data directory when shutting down
def remove_data_directory():
    try:
        shutil.rmtree("data")
        logging.info("Data directory removed successfully!")
    except Exception as e:
        logging.error(f"Couldn't remove the data directory due to exception {e}")
        



In [2]:
 # create spark connection
spark_conn = create_spark_connection()

2025-02-21 13:20:39,807 - INFO - Spark connection created successfully!


In [3]:
spark_df = connect_to_kafka(spark_conn)
transformed_df = spark_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")
transformed_df.printSchema()

2025-02-21 13:20:41,065 - INFO - kafka dataframe created successfully


root
 |-- key: string (nullable = true)
 |-- value: string (nullable = true)



In [4]:
json_schema = StructType([
            StructField('time', TimestampNTZType(), True), \
            StructField('open', DoubleType(), True), \
            StructField('high', DoubleType(), True), \
            StructField('low', DoubleType(), True), \
            StructField('close', DoubleType(), True)
        ])
value_df = transformed_df.withColumn("value", from_json(transformed_df["value"], json_schema)).select("value.*") 
value_df.printSchema()

root
 |-- time: timestamp_ntz (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)



In [5]:
result_df = process(value_df)

root
 |-- time: timestamp_ntz (nullable = true)
 |-- open_close_price: double (nullable = true)



In [ ]:
streaming_query = (result_df.writeStream
                                .foreachBatch(save)
                                .start())

2025-02-21 13:20:52,078 - INFO - Callback Server Starting
2025-02-21 13:20:52,081 - INFO - Socket listening on ('127.0.0.1', 53438)


2025-02-21 13:20:53,957 - INFO - Python Server ready to receive messages
2025-02-21 13:20:53,957 - INFO - Received command c on object id p0


+----+----------------+
|time|open_close_price|
+----+----------------+
+----+----------------+



2025-02-21 13:21:06,180 - INFO - Received command c on object id p0
2025-02-21 13:21:06,523 - ERROR - There was an exception while executing the Python Proxy on the Python Side.
Traceback (most recent call last):
  File "d:\Program_setup\Python\Python312\Lib\site-packages\py4j\clientserver.py", line 617, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Program_setup\Python\Python312\Lib\site-packages\pyspark\sql\utils.py", line 120, in call
    raise e
  File "d:\Program_setup\Python\Python312\Lib\site-packages\pyspark\sql\utils.py", line 117, in call
    self.func(DataFrame(jdf, wrapped_session_jdf), batch_id)
  File "C:\Users\Hl10a\AppData\Local\Temp\ipykernel_7564\1751305016.py", line 74, in save
    df_batch.write.parquet(output_dir, mode="append")
  File "d:\Program_setup\Python\Python312\Lib\site-packages\pyspark\sql\readwriter.py", line 1721, in parquet
    self._jwrite.parquet

In [ ]:

try:
    streaming_query.awaitTermination()
finally:
    remove_data_directory()

2025-02-21 13:19:22,897 - INFO - Callback Server Starting
2025-02-21 13:19:22,900 - INFO - Socket listening on ('127.0.0.1', 53317)
2025-02-21 13:19:25,067 - INFO - Python Server ready to receive messages
2025-02-21 13:19:25,067 - INFO - Received command c on object id p0
2025-02-21 13:19:26,294 - ERROR - There was an exception while executing the Python Proxy on the Python Side.
Traceback (most recent call last):
  File "d:\Program_setup\Python\Python312\Lib\site-packages\py4j\clientserver.py", line 617, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Program_setup\Python\Python312\Lib\site-packages\pyspark\sql\utils.py", line 120, in call
    raise e
  File "d:\Program_setup\Python\Python312\Lib\site-packages\pyspark\sql\utils.py", line 117, in call
    self.func(DataFrame(jdf, wrapped_session_jdf), batch_id)
  File "C:\Users\Hl10a\AppData\Local\Temp\ipykernel_20304\1751305016.py",

StreamingQueryException: [STREAM_FAILED] Query [id = dc671a84-3865-4f71-99fc-ce3770f42765, runId = 5c250e09-ed9c-482b-a7cd-66006858f34a] terminated with exception: An exception was raised by the Python Proxy. Return Message: Traceback (most recent call last):
  File "d:\Program_setup\Python\Python312\Lib\site-packages\py4j\clientserver.py", line 617, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Program_setup\Python\Python312\Lib\site-packages\pyspark\sql\utils.py", line 120, in call
    raise e
  File "d:\Program_setup\Python\Python312\Lib\site-packages\pyspark\sql\utils.py", line 117, in call
    self.func(DataFrame(jdf, wrapped_session_jdf), batch_id)
  File "C:\Users\Hl10a\AppData\Local\Temp\ipykernel_20304\1751305016.py", line 74, in save
    df_batch.write.parquet(output_dir, mode="append")
  File "d:\Program_setup\Python\Python312\Lib\site-packages\pyspark\sql\readwriter.py"